<div class="alert alert-block alert-info">
LINMA2415 - Project - Question 13

- Colson Adeline 19752000
    
- Lhernould Benoit 77102200 

- El Bared Mary Jo 48212000 
  
    
</div>

<div class="alert alert-block alert-info">
Load Data
</div>

In [6]:
using JuMP
using Gurobi
using CSV 
using DataFrames

In [7]:
df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]
# println(size(D_t))


file_path = "capacity_pv.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_pv = Array(df)
println(first(capacity_pv, 10))
    
file_path = "capacity_wind.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_wind = Array(df)
println(first(capacity_wind, 10))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.067, 0.218]
[0.12594, 0.123693, 0.124232, 0.13045, 0.140045, 0.136766, 0.129185, 0.135791, 0.152821, 0.14136]


<div class="alert alert-block alert-info">
Constants and data
</div>

In [8]:
# Constants and Data
T = 8760
lambda = 5000  
cost_of_debt = 0.04
cost_of_equity = 0.07
corporate_tax = 0.30
economic_life = 20

capacity = Dict(
    "Coal" => 1,
    "CCGT" => 1,
    "OCGT" => 1,
    "Onshore Wind" => capacity_wind,
    "Offshore Wind" => capacity_wind,
    "PV" => capacity_pv
)

# Technologies data
technologies = Dict(
    "Coal" => (capex=2000*1000, om=0.03, debt_ratio=0.62, heat_rate=2.4, EA=175.2252157*1000, price =5, C=12),
    "CCGT" => (capex=950*1000, om=0.03, debt_ratio=0.56, heat_rate=1.62, EA=85.60445144*1000, price =30, C=48.6),
    "OCGT" => (capex=700*1000, om=0.03, debt_ratio=0.6, heat_rate=2.5, EA=61.91153839*1000, price =30, C=75),
    "Onshore Wind" => (capex=700*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=58.99797384*1000, price =0, C=0),
    "Offshore Wind" => (capex=1300*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=109.5676657*1000, price =0, C=0),
    "PV" => (capex=400*1000, om=0.03, debt_ratio=0.8, heat_rate=0, EA=32.04823387*1000, price =0, C=0)
)

#Battery data
F_b=6*1000
A_b=1.620627574*1000
eta=0.9
capex_b= 25000



25000

<div class="alert alert-block alert-info">
Question 13
</div>

In [9]:
model_2= Model(Gurobi.Optimizer)


@variable(model_2, k[g in keys(technologies)] >= 0)  # Installed capacity
@variable(model_2, p[g in keys(technologies), t=1:T] >= 0)  # Hourly production
@variable(model_2, d[t=1:T] >=0)
#For Battery
@variable(model_2, k_b>=0 ) 
@variable(model_2, e[1:T] >= 0)
@variable(model_2, p_in[1:T] >=0)
@variable(model_2, p_out[1:T] >=0)


@objective(model_2, Min, sum((technologies[g].om * technologies[g].capex + technologies[g].EA )*k[g] + sum(technologies[g].C*p[g,t]  for t in 1:T) for g in keys(technologies))+ (A_b + F_b)* k_b - sum( lambda * d[t] for t in 1:T))


@constraint(model_2, [t in 1:T], d[t]<=D_t[t])
#@constraint(model_2, [t in 1:T], -d[t] + p_out[t] - p_in[t]  + sum( p[g,t] for g in keys(technologies))==0)
@constraint(model_2, price[t in 1:T], -d[t] + p_out[t] - p_in[t]  + sum( p[g,t] for g in keys(technologies))==0)
@constraint(model_2, [t in 1:T], e[t]<= 4*k_b)
@constraint(model_2, [t in 1:T], p_in[t]<=k_b)
@constraint(model_2, [t in 1:T], p_out[t]<=k_b)
@constraint(model_2,  e[1]==0)
@constraint(model_2, [t in 2:T], e[t]== e[t-1] + 1*(sqrt(eta)* p_in[t-1] - 1/sqrt(eta) * p_out[t-1]))
for g in ["Coal", "CCGT", "OCGT"]
    @constraint(model_2, [t in 1:T], p[g, t] <= k[g])
end
for t in 1:T
    @constraint(model_2, p["Onshore Wind", t] <= capacity_wind[t]*k["Onshore Wind"])
    @constraint(model_2, p["Offshore Wind", t] <= capacity_wind[t]*k["Offshore Wind"])
    @constraint(model_2, p["PV", t] <= capacity_pv[t]*k["PV"])
end

optimize!(model_2)
# print("Variables value, p:",JuMP.value.(p))
print("Variables value, k:",JuMP.value.(k))
print("Variables value, k_b:",JuMP.value.(k_b))
# print("Variables value, d:",JuMP.value.(d))
print("Objective value: ", objective_value(model_2))
print(keys(technologies))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-07
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 105120 rows, 87607 columns and 276211 nonzeros
Model fingerprint: 0xcb029474
Coefficient statistics:
  Matrix range     [1e-03, 4e+00]
  Objective range  [1e+01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 1e+05]
Presolve removed 12869 rows and 4108 columns
Presolve time: 0.39s
Presolved: 92251 rows, 83499 columns, 259233 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.07s

Barrier statistics:
 Dense cols : 7
 AA' NZ     : 1.933e+05
 Factor NZ  : 1.344e+06 (roughly 80 MB of memory)
 Factor Ops : 2.170e+07 (less than 1 second per iteration)
 Threads    :

<div class="alert alert-block alert-info">
Results
</div>

In [10]:
println("\n")
println("Point I")
println("Optimal capacities (MW):")
total_production = 0
production_by_tech = Dict(g => 0.0 for g in keys(technologies))
for g in keys(technologies)
    cap = value(k[g])  # This ensures 'cap' remains a floating-point number
    println("$(g): $(cap) MW")
    for t in 1:T
        production_by_tech[g] += value(p[g, t])  # Accumulate as floating-point values
        total_production += value(p[g, t])  # Keep total production as floating-point
    end
end
    cap = value(k_b)  # This ensures 'cap' remains a floating-point number
    println("Battery: $(cap) MW")

println("\n")

println("Point II")
println("Share of each technology in effective production:")
for g in keys(technologies)
    share = production_by_tech[g] / total_production
    println("$(g): $(share * 100) %")
end
println("\n")


println("Point III")
println("Total Production: $(total_production) MWh")
total_cost = capex_b*value(k_b) + sum((technologies[g].EA+technologies[g].capex)*value.(k[g])+sum((technologies[g].C)*value.(p[g,i]) for i in 1:T)  for g in keys(technologies))
println("Total Investment and Production Cost: €$(total_cost)")
println("\n")



println("Point IV")
# CO2 Emissions data
emissions = Dict(
    "Coal" => 1.4,
    "CCGT" => 0.5,
    "OCGT" => 0.6,
    "Onshore Wind" => 0.0,
    "Offshore Wind" => 0.0,
    "PV" => 0.0
)

# Calculating CO2 emissionys
total_co2_emissions = sum(emissions[g] * production_by_tech[g] for g in keys(production_by_tech))
println("\nTotal CO2 Emissions: $(total_co2_emissions) tons")
println("\n")


println("Point V")
load_curtailment = sum(max(0, D_t[t] - value(d[t])) for t in 1:T)
println("Total Load Curtailment: $(load_curtailment) MWh")
println("\n")


println("Point VI")
# Profit calculation per technology
# profit_by_tech = Dict()
# for g in keys(technologies)
#     revenue = lambda * production_by_tech[g]  # Assuming all production is sold at threshold price λ
#     cost = technologies[g].capex * value(k[g]) + technologies[g].om * technologies[g].capex * value(k[g]) / economic_life
#     profit_by_tech[g] = revenue - cost
# end

# println("\nProfit by Technology:")
# for g in keys(profit_by_tech)
#     println("$(g): €$(profit_by_tech[g])")
# end
profit = Dict(g => 0.0 for g in keys(technologies))
prices = [JuMP.dual(price[i]) for i in 1:T]

for g in keys(technologies)
    profit[g] = -(technologies[g].EA + technologies[g].capex) * value(k[g]) - 
                 sum((technologies[g].C - prices[i]) * value(p[g, i]) for i in 1:T)
end

for g in keys(technologies)
    println("Profit for technology $g: $(profit[g])")
end



Point I
Optimal capacities (MW):
Coal: 40421.38013380037 MW
CCGT: 11865.964179474213 MW
Offshore Wind: 0.0 MW
OCGT: 11783.684839965312 MW
PV: 0.0 MW
Onshore Wind: 57201.20980201635 MW
Battery: 69870.31333575856 MW


Point II
Share of each technology in effective production:
Coal: 66.64803127215137 %
CCGT: 5.283326775723322 %
Offshore Wind: 0.0 %
OCGT: 1.3914575242417826 %
PV: 0.0 %
Onshore Wind: 26.67718442789032 %


Point III
Total Production: 4.7343064570062214e8 MWh
Total Investment and Production Cost: €1.598506180573198e11


Point IV

Total CO2 Emissions: 4.582040825572893e8 tons


Point V
Total Load Curtailment: 0.0 MWh


Point VI
Profit for technology Coal: -7.841747745957317e10
Profit for technology CCGT: -1.093448599138539e10
Profit for technology Offshore Wind: -0.0
Profit for technology OCGT: -8.001122006336444e9
Profit for technology PV: -0.0
Profit for technology Onshore Wind: -3.8839621455569115e10
